# 신경망의 학습

## Repeat 노드

In [1]:
import numpy as np

D, N = 8, 7
x = np.random.rand(1, D)
y = np.repeat(x, N, axis=0)
dy = np.random.rand(N, D)
dx = np.sum(dy, axis=0, keepdims=True)

In [2]:
dx

array([[2.9504499 , 3.82376451, 3.09055425, 3.29941786, 3.13397239,
        4.66950442, 3.53232835, 3.83071715]])

## Sum 노드

In [3]:
import numpy as np

D, N = 8, 7
x = np.random.rand(N, D)
y = np.sum(x, axis=0, keepdims=True)

dy = np.random.rand(1, D)
dx = np.repeat(dy, N, axis=0)

In [4]:
dx

array([[0.83168486, 0.27283738, 0.19510327, 0.85806618, 0.38042834,
        0.73717503, 0.42646507, 0.99969617],
       [0.83168486, 0.27283738, 0.19510327, 0.85806618, 0.38042834,
        0.73717503, 0.42646507, 0.99969617],
       [0.83168486, 0.27283738, 0.19510327, 0.85806618, 0.38042834,
        0.73717503, 0.42646507, 0.99969617],
       [0.83168486, 0.27283738, 0.19510327, 0.85806618, 0.38042834,
        0.73717503, 0.42646507, 0.99969617],
       [0.83168486, 0.27283738, 0.19510327, 0.85806618, 0.38042834,
        0.73717503, 0.42646507, 0.99969617],
       [0.83168486, 0.27283738, 0.19510327, 0.85806618, 0.38042834,
        0.73717503, 0.42646507, 0.99969617],
       [0.83168486, 0.27283738, 0.19510327, 0.85806618, 0.38042834,
        0.73717503, 0.42646507, 0.99969617]])

## MatMul 

In [5]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out

    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

In [30]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
a = b
print(a)
id(a) == id(b)

[4 5 6]


True

In [31]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
a[...] = b
print(a)
id(a) == id(b)

[4 5 6]


False

## 시그모이드 계층

In [32]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

## Affine 계층

In [33]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out

    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

## Softmax with Loss 계층

In [34]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmax의 출력
        self.t = None  # 정답 레이블

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 정답 레이블이 원핫 벡터일 경우 정답의 인덱스로 변환
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx

## 가중치 갱신

In [35]:
class SGD:
    '''
    확률적 경사하강법(Stochastic Gradient Descent)
    '''
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]